# MNIST

* 10 kategoriye ayrılmış (0'dan 9'a kadar) elle yazılmış rakamlardan oluşan, siyah-beyaz renkte, 28x28 piksel görüntülerden oluşuyor.
* 60.000 tanesi eğitim için, 10.000 tanesi test için olmak üzere toplamda 70.000 tane görüntü vardır.

<p align = "left">
    <img src="https://i1.wp.com/www.marktechpost.com/wp-content/uploads/2019/10/1_QAAGYDHreoRm4vEArNzTTQ.png?resize=372%2C238&ssl=1" /></p>

* Bu veriseti Keras üzerinde Numpy dizileri şeklinde yüklü halde gelmektedir.

In [1]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

* **train_images** ve **train_labels** modelimizin öğrenmek için kullanacağı eğitim veri setini oluşturmaktadır.
* **test_images** ve **test_labels** üzerinden de modelimizi test edeceğiz.
* Görüntüler Numpy dizisi olarak kodlanmıştır.
* Etiketler 0 ile 9 arasındaki rakamlardan oluşan birer dizidir.
* Görüntüler ve etiketler arasında birebir ilişki vardır.

In [2]:
print("Train Görüntü Seti Boyutu:  ", train_images.shape, "\n"
      "Train Etiket Seti Boyutu:   ", train_labels.shape, "\n"
      "Train Etiket Seti Görünümü: ", train_labels, "\n")

print("Test Görüntü Seti Boyutu:   ", test_images.shape, "\n"
      "Test Etiket Seti Boyutu:    ", test_labels.shape, "\n"
      "Test Etiket Seti Görünümü:  ", test_labels)

Train Görüntü Seti Boyutu:   (60000, 28, 28) 
Train Etiket Seti Boyutu:    (60000,) 
Train Etiket Seti Görünümü:  [5 0 4 ... 5 6 8] 

Test Görüntü Seti Boyutu:    (10000, 28, 28) 
Test Etiket Seti Boyutu:     (10000,) 
Test Etiket Seti Görünümü:   [7 2 1 ... 4 5 6]


* Öncelikle Sinir Ağı'nı eğitim veri seti ile besleyeceğiz.
* Ağ görüntüler ile etiketleri eşleştirmeyi öğrenecek.
* Sonra **test setleri** ile ağı test edeceğiz.

## Ağ Mimarisi

* Derin ağların temel yapı taşı, veri için filtre olarak düşünülebilecek veri işleme modülü olan katmanlardır.
* Veri katmana ham hali girer ve daha kullanışlı formda çıkar.
* Katmanlar kendisine verilen verilerden problemin çözümünde katkıda olacak daha anlamlı gösterimler çıkartmaya çalışırlar.
* Derin öğrenmede öğrenme süreci çoğunlukla basit katmanları üst üste getirerek verinin ilerledikçe daha da arıtılmasını sağlayan yapıdadır.
* Veri ilerledikçe daha da arıtıldığı birbirini takip eden süzgeçlere benzetilebilir.

In [3]:
from keras import models # Model
from keras import layers # Katmanlar

* Modelimiz iki adet birbirini takip eden **Dense²** katmanı içeriyor.
* İkinci (yani son katman) 10 adet çıktı birimi içermektedir.
* Toplamları 1 olan 10 elemanlı olasılık puanları vardır.
* Dizinin her bir elemanı o anki örneğimizdeki sayının 1'den 10'a kadar hangi sınıfa ait olduğunu gösteren bir olasılık değeridir.

In [4]:
network = models.Sequential() #Boş model oluşturuyoruz

network.add(layers.Dense(512, activation="relu", input_shape=(28*28,))) # Layers fonksiyonunu kullanarak dense katmanını ekliyoruz
                                                                        # 512 tane eleman girişi yaptık
                                                                        # Aktivasyon fonksiyonu olarak "relu" girişi yapıyoruz
                                                                        # Boyut olarak da her bir görüntümüzün boyutu 28*28
            
network.add(layers.Dense(10, activation="softmax")) # Sınıflandırıcı katmanını tekrar dense ile ekliyoruz
                                                    # 0'dan 9'a kadar 10 sınıf vardır
                                                    # Sınıflandırma için genellikle "softmax" seçiyoruz

In [5]:
network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Derleme Adımı

Ağımızı eğitime hazır hale getirmek için 3 şeyi daha derleme adımı olarak almamız gerekiyor:

* **Kayıp Fonksiyonu:** Ağızımın eğitim veri üzerinde kndi performansını gözlemlemesi ve böylece kendi kendine doğru yolu bulabilmesi içindir.
* **Eniyileme:** Ağımızın girdisi olan veri ile oluşturduğu kaybı göz önünde bulundurarak kendisini güncelleme mekanizmasıdır.
* **Eğitim ve test süresince takip edilecek metrikler:** Burada sadece doğruluğa (doğru sınıflandırılan görüntülerin toplam görüntü sayısına oranı) odaklanacağız.

In [6]:
# Optimizer olarak "rmsprop" kullanıyoruz
# Kayıp fonksiyonu olarak "categorical_crossentropy" seçiyoruz
# Metrik olarak da en çok tercih edilen "accuracy" kullanıyoruz

network.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

## Girdilerin Hazırlanması

* Eğitime başlamadan önce tüm girdiklerimizdeki değerleri [0,1] aralığına ölçeklendiriyoruz.
* Bu aşamadan önce eğitim veri setimizdeki görüntüler (60000,28,28) şeklinde bir dizide ve her elemanı **uinit8** veri tipinde [0,255] veri aralığında saklanmıştı.
* Eğitim veri setindeki görüntüleri (60000,28,28) **float32** veri tipinde [0,1] aralığında düzenliyoruz.

In [7]:
train_images = train_images.reshape((60000, 28*28))
train_images = train_images.astype("float32") / 255 # Normalizasyon (klasik olarak görüntü işlemede bu yapılır)

test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype("float32") / 255

## Etiketlerin Hazırlanması

* Verileri kategorik olarak dönüştürmemiz gerekiyor.

In [8]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

## Modelin Eğitilmesi

* Artık eğitime başlayabiliriz.
* Bunun için **Keras**'ın **fit** metodunu çağırarak modelimizi eğitim veri setine uyduruyoruz.
* Eğitim süresince iki değer gösterilecek.
* Ağın eğitim veri seti üzerindeki kaybı ve doğruluğunu göreceğiz.

In [9]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 3s 6ms/step - loss: 0.4289 - accuracy: 0.8763
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 0.1099 - accuracy: 0.9681
Epoch 3/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0708 - accuracy: 0.9790
Epoch 4/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0501 - accuracy: 0.9853
Epoch 5/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0350 - accuracy: 0.9898


* Eğitim setinde **0.9895** yani **%98.9**'luk bir doğruluğa ulaştık.
* Şimdi test seti üzetinde deneyeceğiz.

In [10]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

313/313 [==============================] - 0s 821us/step - loss: 0.0659 - accuracy: 0.9804


* Eğitim setinin bir tık altında doğruluk elde ettik.

In [11]:
print("test_loss:", test_loss, "\n"
      "test_acc:", test_acc)

test_loss: 0.06588851660490036 
test_acc: 0.980400025844574
